# Deep Analysis: Sniper Optimization Runs 32 & 33

**Run 32**: 115 model trials, original search spaces  
**Run 33**: 75 model trials, expanded search spaces, fixed Optuna objective  

**Bet types**: away_win, btts, cards, corners, fouls, home_win, over25, shots, under25

In [ ]:
import json
import yaml
from pathlib import Path
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from scipy import stats
try:
    display
except NameError:
    display = print  # fallback for non-Jupyter execution

pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', '{:.3f}'.format)
sns.set_theme(style='whitegrid', palette='colorblind')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['figure.dpi'] = 100

BET_TYPES = ['away_win', 'btts', 'cards', 'corners', 'fouls', 'home_win', 'over25', 'shots', 'under25']
# Resolve project root robustly (works from notebook CWD or project root)
_cwd = Path.cwd()
_PROJECT_ROOT = _cwd if (_cwd / 'data' / 'artifacts').exists() else _cwd / '..'
_PROJECT_ROOT = _PROJECT_ROOT.resolve()
RUNS = {32: _PROJECT_ROOT / 'data' / 'artifacts' / 'sniper-all-results-32',
        33: _PROJECT_ROOT / 'data' / 'artifacts' / 'sniper-all-results-33'}
print(f'Project root: {_PROJECT_ROOT}')
print(f'Run 32 exists: {RUNS[32].exists()}, Run 33 exists: {RUNS[33].exists()}')

## 1. Data Loading & Setup

In [ ]:
def load_latest_json(directory: Path, prefix: str) -> dict | None:
    """Load the latest JSON file matching prefix (by filename timestamp)."""
    files = sorted(directory.glob(f'{prefix}*.json'))
    if not files:
        return None
    with open(files[-1]) as f:
        return json.load(f)

def load_yaml(path: Path) -> dict | None:
    if not path.exists():
        return None
    with open(path) as f:
        return yaml.safe_load(f)

# Load all data
sniper = {}   # sniper[run][bet_type] = dict
fparams = {}  # fparams[run][bet_type] = dict
yamls = {}    # yamls[run][bet_type] = dict

for run_id, run_dir in RUNS.items():
    sniper[run_id] = {}
    fparams[run_id] = {}
    yamls[run_id] = {}
    for bt in BET_TYPES:
        sniper[run_id][bt] = load_latest_json(run_dir, f'sniper_{bt}_')
        fparams[run_id][bt] = load_latest_json(run_dir, f'feature_params_{bt}_')
        yamls[run_id][bt] = load_yaml(run_dir / 'feature_params' / f'{bt}.yaml')

# Verify loading
for run_id in [32, 33]:
    loaded = [bt for bt in BET_TYPES if sniper[run_id][bt] is not None]
    fp_loaded = [bt for bt in BET_TYPES if fparams[run_id][bt] is not None]
    print(f'Run {run_id}: {len(loaded)} sniper results, {len(fp_loaded)} feature params')
    if len(loaded) < 9:
        print(f'  Missing sniper: {set(BET_TYPES) - set(loaded)}')
    if len(fp_loaded) < 9:
        print(f'  Missing fparams: {set(BET_TYPES) - set(fp_loaded)}')

In [ ]:
# Build unified DataFrames

# Sniper summary DataFrame
rows = []
for run_id in [32, 33]:
    for bt in BET_TYPES:
        s = sniper[run_id][bt]
        if s is None:
            continue
        h = s.get('holdout_metrics', {})
        row = {
            'run': run_id, 'bet_type': bt,
            'best_model': s.get('best_model'),
            'n_features': s.get('n_features'),
            'threshold': s.get('best_threshold'),
            'min_odds': s.get('best_min_odds'),
            'max_odds': s.get('best_max_odds'),
            'bt_precision': s.get('precision'),
            'bt_roi': s.get('roi'),
            'bt_n_bets': s.get('n_bets'),
            'holdout_precision': h.get('precision'),
            'holdout_roi': h.get('roi'),
            'holdout_n_bets': h.get('n_bets'),
            'holdout_sharpe': h.get('sharpe'),
            'holdout_sortino': h.get('sortino'),
            'holdout_ece': h.get('ece'),
        }
        rows.append(row)

df_sniper = pd.DataFrame(rows)
print(f'Sniper summary: {len(df_sniper)} rows ({df_sniper.run.nunique()} runs × {df_sniper.bet_type.nunique()} bet types)')
df_sniper.head(4)

In [ ]:
# Feature params summary DataFrame
fp_rows = []
for run_id in [32, 33]:
    for bt in BET_TYPES:
        fp = fparams[run_id][bt]
        if fp is None:
            continue
        row = {'run': run_id, 'bet_type': bt,
               'sharpe': fp.get('sharpe'), 'precision': fp.get('precision'),
               'roi': fp.get('roi'), 'n_bets': fp.get('n_bets'),
               'n_trials': fp.get('n_trials')}
        for k, v in fp.get('best_params', {}).items():
            row[f'param_{k}'] = v
        fp_rows.append(row)

df_fparams = pd.DataFrame(fp_rows)
df_fparams.head(4)

## 2. Feature Parameter Tuning Analysis

In [ ]:
# Search space definitions
SEARCH_SPACES = {
    'elo_k_factor': (10, 50),
    'elo_home_advantage': (25, 250),
    'form_window': (3, 20),
    'ema_span': (3, 20),
    'poisson_lookback': (5, 40),
    'fouls_ema_span': (3, 20),
    'cards_ema_span': (3, 20),
    'shots_ema_span': (3, 20),
    'corners_ema_span': (3, 20),
}

# Boundary analysis: flag params within 10% of boundary
boundary_rows = []
for run_id in [32, 33]:
    for bt in BET_TYPES:
        fp = fparams[run_id][bt]
        if fp is None:
            continue
        # Use search_space from the JSON if available, else global
        ss = fp.get('search_space', {})
        for param, value in fp.get('best_params', {}).items():
            if param in ss:
                lo, hi = ss[param][0], ss[param][1]
            elif param in SEARCH_SPACES:
                lo, hi = SEARCH_SPACES[param]
            else:
                continue
            rng = hi - lo
            near_lo = (value - lo) / rng <= 0.1
            near_hi = (hi - value) / rng <= 0.1
            boundary_rows.append({
                'run': run_id, 'bet_type': bt, 'param': param,
                'value': value, 'lo': lo, 'hi': hi,
                'pct_of_range': (value - lo) / rng,
                'at_boundary': 'LOW' if near_lo else ('HIGH' if near_hi else 'OK')
            })

df_boundary = pd.DataFrame(boundary_rows)
flagged = df_boundary[df_boundary.at_boundary != 'OK'].sort_values(['param', 'bet_type', 'run'])
print(f'=== Parameters at boundary (within 10% of min/max) ===')
print(f'{len(flagged)} cases flagged out of {len(df_boundary)} total\n')
if len(flagged) > 0:
    display(flagged[['run', 'bet_type', 'param', 'value', 'lo', 'hi', 'at_boundary']])

In [ ]:
# Boundary visualization: best param values vs search space bounds
core_params = ['elo_k_factor', 'form_window', 'ema_span', 'poisson_lookback']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
for ax, param in zip(axes.flat, core_params):
    lo, hi = SEARCH_SPACES[param]
    for run_id, marker in [(32, 'o'), (33, 's')]:
        vals, labels = [], []
        for bt in BET_TYPES:
            fp = fparams[run_id][bt]
            if fp and param in fp.get('best_params', {}):
                vals.append(fp['best_params'][param])
                labels.append(bt)
        if vals:
            ax.scatter(labels, vals, marker=marker, s=80, label=f'Run {run_id}', zorder=3)
    ax.axhline(lo, color='red', ls='--', alpha=0.5, label='Search bounds')
    ax.axhline(hi, color='red', ls='--', alpha=0.5)
    # 10% boundary zones
    rng = hi - lo
    ax.axhspan(lo, lo + 0.1 * rng, color='red', alpha=0.08)
    ax.axhspan(hi - 0.1 * rng, hi, color='red', alpha=0.08)
    ax.set_title(param)
    ax.tick_params(axis='x', rotation=45)
    ax.legend(fontsize=8)

fig.suptitle('Feature Parameter Best Values vs Search Space Bounds', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Trial convergence: Sharpe/ROI across trials per bet type
fig, axes = plt.subplots(3, 3, figsize=(16, 12))
for ax, bt in zip(axes.flat, BET_TYPES):
    for run_id, color in [(32, 'C0'), (33, 'C1')]:
        fp = fparams[run_id][bt]
        if fp is None or 'all_trials' not in fp:
            continue
        trials = fp['all_trials']
        trial_nums = [t['number'] for t in trials]
        rois = [t.get('roi', 0) for t in trials]
        ax.plot(trial_nums, rois, marker='.', ms=4, alpha=0.7, color=color, label=f'R{run_id}')
        # Running best
        running_best = np.maximum.accumulate(rois)
        ax.plot(trial_nums, running_best, ls='--', color=color, alpha=0.5)
    ax.set_title(bt, fontsize=10)
    ax.set_xlabel('Trial #', fontsize=8)
    ax.set_ylabel('ROI %', fontsize=8)
    ax.legend(fontsize=7)

fig.suptitle('Feature Param Optimization Convergence (solid=trial ROI, dashed=running best)', fontsize=13)
plt.tight_layout()
plt.show()

In [ ]:
# Parameter distributions: violin plots of trial values per param
trial_param_rows = []
for run_id in [32, 33]:
    for bt in BET_TYPES:
        fp = fparams[run_id][bt]
        if fp is None or 'all_trials' not in fp:
            continue
        for trial in fp['all_trials']:
            for param, val in trial.get('params', {}).items():
                trial_param_rows.append({
                    'run': run_id, 'bet_type': bt, 'param': param, 'value': val,
                    'trial': trial['number']
                })

df_trial_params = pd.DataFrame(trial_param_rows)

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
for ax, param in zip(axes.flat, core_params):
    subset = df_trial_params[df_trial_params.param == param]
    if subset.empty:
        ax.set_title(f'{param} (no data)')
        continue
    sns.violinplot(data=subset, x='bet_type', y='value', hue='run',
                   split=True, inner='quart', ax=ax, palette='Set2')
    lo, hi = SEARCH_SPACES.get(param, (None, None))
    if lo is not None:
        ax.axhline(lo, color='red', ls=':', alpha=0.4)
        ax.axhline(hi, color='red', ls=':', alpha=0.4)
    ax.set_title(param)
    ax.tick_params(axis='x', rotation=45)

fig.suptitle('Parameter Distributions Across Optuna Trials', fontsize=13)
plt.tight_layout()
plt.show()

In [ ]:
# Cross-bet-type param heatmap: best params per bet type
param_cols = [c for c in df_fparams.columns if c.startswith('param_')]

for run_id in [32, 33]:
    sub = df_fparams[df_fparams.run == run_id].set_index('bet_type')[param_cols].dropna(axis=1, how='all')
    # Normalize each param 0-1 for heatmap comparability
    sub_norm = (sub - sub.min()) / (sub.max() - sub.min() + 1e-9)
    sub_norm.columns = [c.replace('param_', '') for c in sub_norm.columns]

    # Build raw annotation array
    raw = sub.copy()
    raw.columns = [c.replace('param_', '') for c in raw.columns]
    annot = raw.fillna(0).values

    fig, ax = plt.subplots(figsize=(10, 6))
    sns.heatmap(sub_norm, annot=annot, fmt='.0f', cmap='YlOrRd', ax=ax, linewidths=0.5)
    ax.set_title(f'Run {run_id}: Best Feature Params (color=normalized, text=raw value)')
    plt.tight_layout()
    plt.show()

In [ ]:
# Run 32 → 33 delta table
delta_rows = []
for bt in BET_TYPES:
    fp32 = fparams[32].get(bt)
    fp33 = fparams[33].get(bt)
    if fp32 is None or fp33 is None:
        continue
    p32 = fp32.get('best_params', {})
    p33 = fp33.get('best_params', {})
    all_params = set(p32.keys()) | set(p33.keys())
    for param in sorted(all_params):
        v32 = p32.get(param)
        v33 = p33.get(param)
        if v32 is not None and v33 is not None:
            delta = v33 - v32
            pct_delta = delta / (abs(v32) + 1e-9) * 100
        else:
            delta = None
            pct_delta = None
        delta_rows.append({
            'bet_type': bt, 'param': param,
            'run32': v32, 'run33': v33,
            'delta': delta, 'pct_change': pct_delta
        })

df_delta = pd.DataFrame(delta_rows)
print('=== Feature Parameter Changes: Run 32 → 33 ===')
# Show only params that changed
changed = df_delta[df_delta.delta.notna() & (df_delta.delta.abs() > 0)]
display(changed.sort_values('pct_change', ascending=False, key=abs))

In [ ]:
# Recommendation table: per param, per bet type
rec_rows = []
for _, row in df_boundary.iterrows():
    if row['at_boundary'] == 'LOW':
        action = f"EXPAND lower bound below {row['lo']}"
    elif row['at_boundary'] == 'HIGH':
        action = f"EXPAND upper bound above {row['hi']}"
    else:
        action = 'Keep current range'
    rec_rows.append({
        'run': row['run'], 'bet_type': row['bet_type'], 'param': row['param'],
        'value': row['value'], 'range': f"[{row['lo']}, {row['hi']}]",
        'position': f"{row['pct_of_range']:.0%}",
        'recommendation': action
    })

df_rec = pd.DataFrame(rec_rows)
# Show run 33 recommendations (most recent)
print('=== Run 33 Parameter Range Recommendations ===')
display(df_rec[df_rec.run == 33].sort_values(['param', 'bet_type'])
        [['bet_type', 'param', 'value', 'range', 'position', 'recommendation']])

## 3. Model Hyperparameter Tuning Analysis

In [ ]:
# Best model comparison: run 32 vs 33
model_comp_rows = []
for bt in BET_TYPES:
    for run_id in [32, 33]:
        s = sniper[run_id].get(bt)
        if s is None:
            continue
        wf = s.get('walkforward', {})
        wf_best = wf.get('best_model_wf', s.get('best_model'))
        model_comp_rows.append({
            'bet_type': bt, 'run': run_id,
            'best_model': s.get('best_model'),
            'wf_best_model': wf_best,
            'threshold': s.get('best_threshold'),
            'bt_roi': s.get('roi'),
            'holdout_roi': s.get('holdout_metrics', {}).get('roi'),
        })

df_models = pd.DataFrame(model_comp_rows)
print('=== Best Model per Bet Type ===')
pivot = df_models.pivot(index='bet_type', columns='run', values=['best_model', 'wf_best_model', 'holdout_roi'])
display(pivot)

In [ ]:
# Hyperparameter comparison: extract best_params from sniper JSONs
hp_rows = []
for bt in BET_TYPES:
    for run_id in [32, 33]:
        s = sniper[run_id].get(bt)
        if s is None:
            continue
        bp = s.get('best_params', {})
        row = {'bet_type': bt, 'run': run_id, 'model': s.get('best_model')}
        row.update(bp)
        hp_rows.append(row)

df_hp = pd.DataFrame(hp_rows)
print('=== Model Hyperparameters ===')
display(df_hp)

In [ ]:
# Per-algorithm walkforward ROI: bar chart
wf_rows = []
for run_id in [32, 33]:
    for bt in BET_TYPES:
        s = sniper[run_id].get(bt)
        if s is None:
            continue
        wf_summary = s.get('walkforward', {}).get('summary', {})
        for model_name, metrics in wf_summary.items():
            wf_rows.append({
                'run': run_id, 'bet_type': bt, 'model': model_name,
                'avg_roi': metrics.get('avg_roi', 0),
                'std_roi': metrics.get('std_roi', 0),
                'avg_precision': metrics.get('avg_precision', 0),
                'total_bets': metrics.get('total_bets', 0),
            })

df_wf = pd.DataFrame(wf_rows)

# Plot per-algorithm average ROI for run 33
fig, axes = plt.subplots(3, 3, figsize=(18, 14))
for ax, bt in zip(axes.flat, BET_TYPES):
    sub = df_wf[(df_wf.bet_type == bt)].copy()
    if sub.empty:
        ax.set_title(f'{bt} (no data)')
        continue
    pivot_data = sub.pivot_table(index='model', columns='run', values='avg_roi', aggfunc='first')
    pivot_data.plot(kind='bar', ax=ax, width=0.7)
    ax.axhline(0, color='black', lw=0.5)
    ax.set_title(bt, fontsize=11)
    ax.set_ylabel('Avg ROI %')
    ax.tick_params(axis='x', rotation=45)
    ax.legend(title='Run', fontsize=7)

fig.suptitle('Walk-Forward Average ROI per Algorithm per Bet Type', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Ensemble vs single model analysis
single_models = ['lightgbm', 'catboost', 'xgboost']
ensemble_models = ['stacking', 'average', 'agreement']

ens_rows = []
for run_id in [32, 33]:
    for bt in BET_TYPES:
        sub = df_wf[(df_wf.run == run_id) & (df_wf.bet_type == bt)]
        singles = sub[sub.model.isin(single_models)]
        ensembles = sub[sub.model.isin(ensemble_models)]
        if singles.empty or ensembles.empty:
            continue
        best_single = singles.loc[singles.avg_roi.idxmax()]
        best_ens = ensembles.loc[ensembles.avg_roi.idxmax()]
        ens_rows.append({
            'run': run_id, 'bet_type': bt,
            'best_single': best_single['model'],
            'single_roi': best_single['avg_roi'],
            'best_ensemble': best_ens['model'],
            'ensemble_roi': best_ens['avg_roi'],
            'ensemble_wins': best_ens['avg_roi'] > best_single['avg_roi'],
            'roi_delta': best_ens['avg_roi'] - best_single['avg_roi'],
        })

df_ens = pd.DataFrame(ens_rows)
print('=== Ensemble vs Single Model (Run 33) ===')
display(df_ens[df_ens.run == 33].sort_values('roi_delta', ascending=False))
print(f"\nEnsemble wins in {df_ens[df_ens.run == 33].ensemble_wins.sum()}/{len(df_ens[df_ens.run == 33])} bet types")

## 4. SHAP & Feature Importance Analysis

In [ ]:
# Extract top features per bet type per run
top_features = {}  # top_features[run][bt] = list of (feature, importance)
for run_id in [32, 33]:
    top_features[run_id] = {}
    for bt in BET_TYPES:
        s = sniper[run_id].get(bt)
        if s is None:
            continue
        shap = s.get('shap_analysis', {})
        tf = shap.get('top_features', [])
        top_features[run_id][bt] = [(f['feature'], f['importance']) for f in tf[:20]]

In [ ]:
# Side-by-side top 20 feature importance bar charts
fig, axes = plt.subplots(3, 3, figsize=(20, 18))
for ax, bt in zip(axes.flat, BET_TYPES):
    tf32 = dict(top_features[32].get(bt, []))
    tf33 = dict(top_features[33].get(bt, []))
    all_feats = list(dict.fromkeys(list(tf33.keys())[:15] + list(tf32.keys())[:15]))[:15]
    if not all_feats:
        ax.set_title(f'{bt} (no SHAP data)')
        continue
    y = np.arange(len(all_feats))
    vals32 = [tf32.get(f, 0) for f in all_feats]
    vals33 = [tf33.get(f, 0) for f in all_feats]
    ax.barh(y + 0.15, vals33, height=0.3, label='R33', color='C1', alpha=0.8)
    ax.barh(y - 0.15, vals32, height=0.3, label='R32', color='C0', alpha=0.8)
    ax.set_yticks(y)
    ax.set_yticklabels([f[:25] for f in all_feats], fontsize=7)
    ax.set_title(bt, fontsize=11)
    ax.legend(fontsize=7)
    ax.invert_yaxis()

fig.suptitle('Top 15 Feature Importances (SHAP) per Bet Type', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Feature stability: rank correlation between runs
stability_rows = []
for bt in BET_TYPES:
    tf32 = top_features[32].get(bt, [])
    tf33 = top_features[33].get(bt, [])
    if len(tf32) < 5 or len(tf33) < 5:
        continue
    feats32 = [f for f, _ in tf32]
    feats33 = [f for f, _ in tf33]
    common = set(feats32[:20]) & set(feats33[:20])
    
    # Spearman on common features
    if len(common) >= 3:
        ranks32 = {f: i for i, f in enumerate(feats32)}
        ranks33 = {f: i for i, f in enumerate(feats33)}
        common_list = sorted(common)
        r32 = [ranks32[f] for f in common_list]
        r33 = [ranks33[f] for f in common_list]
        rho, pval = stats.spearmanr(r32, r33)
    else:
        rho, pval = np.nan, np.nan
    
    stability_rows.append({
        'bet_type': bt,
        'top20_overlap': len(common),
        'overlap_pct': len(common) / 20 * 100,
        'spearman_rho': rho,
        'p_value': pval,
    })

df_stability = pd.DataFrame(stability_rows)
print('=== Feature Ranking Stability (Run 32 vs 33) ===')
display(df_stability.sort_values('spearman_rho', ascending=False))

In [ ]:
# Cross-bet-type feature overlap heatmap (run 33)
all_top20 = {}
for bt in BET_TYPES:
    tf = top_features[33].get(bt, [])
    all_top20[bt] = set(f for f, _ in tf[:20])

overlap_matrix = pd.DataFrame(index=BET_TYPES, columns=BET_TYPES, dtype=float)
for bt1 in BET_TYPES:
    for bt2 in BET_TYPES:
        s1, s2 = all_top20.get(bt1, set()), all_top20.get(bt2, set())
        if s1 and s2:
            overlap_matrix.loc[bt1, bt2] = len(s1 & s2)
        else:
            overlap_matrix.loc[bt1, bt2] = 0

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(overlap_matrix.astype(float), annot=True, fmt='.0f', cmap='YlGn',
            ax=ax, vmin=0, vmax=20, linewidths=0.5)
ax.set_title('Top-20 Feature Overlap Between Bet Types (Run 33)', fontsize=13)
plt.tight_layout()
plt.show()

In [ ]:
# Low importance features: candidates for removal
low_imp_union = set()
low_imp_counts = defaultdict(int)  # feature -> count of bet types where it's low importance

for run_id in [33]:  # Focus on latest run
    for bt in BET_TYPES:
        s = sniper[run_id].get(bt)
        if s is None:
            continue
        shap = s.get('shap_analysis', {})
        low = shap.get('low_importance_features', [])
        for f in low:
            low_imp_counts[f] += 1
            low_imp_union.add(f)

# Features that are low importance across many bet types
print(f'Total unique low-importance features: {len(low_imp_union)}')
print(f'\n=== Features with low importance in 7+ bet types (safe removal candidates) ===')
removal_candidates = {f: c for f, c in low_imp_counts.items() if c >= 7}
for f, c in sorted(removal_candidates.items(), key=lambda x: -x[1]):
    print(f'  {f}: low in {c}/9 bet types')

print(f'\n=== Features with low importance in 5-6 bet types (review candidates) ===')
review_candidates = {f: c for f, c in low_imp_counts.items() if 5 <= c <= 6}
for f, c in sorted(review_candidates.items(), key=lambda x: -x[1]):
    print(f'  {f}: low in {c}/9 bet types')

In [ ]:
# Feature interactions: top 10 per bet type
interaction_rows = []
for run_id in [33]:
    for bt in BET_TYPES:
        s = sniper[run_id].get(bt)
        if s is None:
            continue
        shap = s.get('shap_analysis', {})
        interactions = shap.get('feature_interactions', [])
        for ix in interactions[:10]:
            interaction_rows.append({
                'bet_type': bt,
                'feature1': ix.get('feature1', ''),
                'feature2': ix.get('feature2', ''),
                'strength': ix.get('interaction_strength', 0),
            })

df_interactions = pd.DataFrame(interaction_rows)
if not df_interactions.empty:
    # Cross-bet-type interaction patterns
    df_interactions['pair'] = df_interactions.apply(
        lambda r: tuple(sorted([r['feature1'], r['feature2']])), axis=1
    )
    pair_counts = df_interactions.groupby('pair').agg(
        n_bet_types=('bet_type', 'nunique'),
        avg_strength=('strength', 'mean'),
        bet_types=('bet_type', lambda x: ', '.join(sorted(set(x))))
    ).sort_values('n_bet_types', ascending=False)

    print('=== Feature Interactions Appearing in Multiple Bet Types ===')
    display(pair_counts[pair_counts.n_bet_types >= 2].head(15))
else:
    print('No interaction data available')

## 5. Business Metrics & Strategy Analysis

In [ ]:
# Comprehensive comparison table
print('=== Precision / ROI / Sharpe / Sortino: Run 32 vs 33 ===')
metrics_cols = ['bet_type', 'run', 'best_model', 'bt_precision', 'bt_roi', 'bt_n_bets',
                'holdout_precision', 'holdout_roi', 'holdout_n_bets',
                'holdout_sharpe', 'holdout_sortino', 'holdout_ece']
display(df_sniper[metrics_cols].sort_values(['bet_type', 'run']))

In [ ]:
# Overfitting gap: backtest ROI vs holdout ROI
df_sniper['overfit_gap'] = df_sniper['bt_roi'] - df_sniper['holdout_roi']

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
for ax, run_id in zip(axes, [32, 33]):
    sub = df_sniper[df_sniper.run == run_id].copy().reset_index(drop=True)
    x = np.arange(len(sub))
    w = 0.35
    ax.bar(x - w/2, sub['bt_roi'], w, label='Backtest ROI', color='C0', alpha=0.8)
    ax.bar(x + w/2, sub['holdout_roi'].fillna(0), w, label='Holdout ROI', color='C3', alpha=0.8)
    ax.set_xticks(x)
    ax.set_xticklabels(sub['bet_type'], rotation=45, ha='right')
    ax.axhline(0, color='black', lw=0.5)
    ax.set_title(f'Run {run_id}: Backtest vs Holdout ROI')
    ax.set_ylabel('ROI %')
    ax.legend()
    # Annotate large gaps
    for i, row in sub.iterrows():
        gap = row['overfit_gap']
        if pd.notna(gap) and abs(gap) > 5:
            y_pos = max(row['bt_roi'], row.get('holdout_roi', 0) or 0) + 2
            color = 'red' if gap > 20 else 'orange'
            ax.text(i, y_pos, f'gap={gap:.0f}%', ha='center', fontsize=7, color=color)

plt.tight_layout()
plt.show()

In [ ]:
# Walk-forward stability: std of ROI across folds
wf_stability_rows = []
for run_id in [32, 33]:
    for bt in BET_TYPES:
        s = sniper[run_id].get(bt)
        if s is None:
            continue
        wf = s.get('walkforward', {}).get('summary', {})
        best_model = s.get('best_model', '')
        metrics = wf.get(best_model, {})
        avg_roi = metrics.get('avg_roi', 0)
        std_roi = metrics.get('std_roi', 0)
        cv = std_roi / (abs(avg_roi) + 1e-9) if avg_roi != 0 else np.inf
        wf_stability_rows.append({
            'run': run_id, 'bet_type': bt, 'model': best_model,
            'avg_roi': avg_roi, 'std_roi': std_roi, 'cv_coeff': cv,
            'total_bets': metrics.get('total_bets', 0),
        })

df_wf_stability = pd.DataFrame(wf_stability_rows)
print('=== Walk-Forward Stability (best model per bet type) ===')
display(df_wf_stability.sort_values(['bet_type', 'run']))

In [ ]:
# Bet volume vs quality tradeoff
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
for ax, run_id in zip(axes, [32, 33]):
    sub = df_sniper[df_sniper.run == run_id].dropna(subset=['holdout_n_bets', 'holdout_roi'])
    ax.scatter(sub['holdout_n_bets'], sub['holdout_roi'], s=100, zorder=3)
    for _, row in sub.iterrows():
        ax.text(row['holdout_n_bets'] + 2, row['holdout_roi'] + 2, row['bet_type'], fontsize=8)
    ax.axhline(0, color='red', ls='--', alpha=0.5)
    ax.set_xlabel('Number of Bets (Holdout)')
    ax.set_ylabel('Holdout ROI %')
    ax.set_title(f'Run {run_id}: Volume vs Quality')

plt.tight_layout()
plt.show()

In [ ]:
# ECE calibration analysis
print('=== Expected Calibration Error (lower is better, <0.05 = well calibrated) ===')
ece_data = df_sniper[['bet_type', 'run', 'holdout_ece']].pivot(
    index='bet_type', columns='run', values='holdout_ece'
)
ece_data['delta'] = ece_data[33] - ece_data[32]
ece_data['well_calibrated_33'] = ece_data[33] < 0.05
display(ece_data.sort_values(33))

In [ ]:
# Portfolio-level analysis
print('=== Portfolio Analysis (All Bet Types Combined) ===')
for run_id in [32, 33]:
    sub = df_sniper[df_sniper.run == run_id].dropna(subset=['holdout_n_bets', 'holdout_roi'])
    total_bets = sub['holdout_n_bets'].sum()
    weighted_roi = (sub['holdout_roi'] * sub['holdout_n_bets']).sum() / total_bets
    profitable = sub[sub.holdout_roi > 0]
    print(f'\nRun {run_id}:')
    print(f'  Total holdout bets: {total_bets:.0f}')
    print(f'  Weighted avg holdout ROI: {weighted_roi:.1f}%')
    print(f'  Profitable markets: {len(profitable)}/{len(sub)}')
    print(f'  Best market: {sub.loc[sub.holdout_roi.idxmax(), "bet_type"]} ({sub.holdout_roi.max():.1f}%)')
    print(f'  Worst market: {sub.loc[sub.holdout_roi.idxmin(), "bet_type"]} ({sub.holdout_roi.min():.1f}%)')

# Correlation between runs
print('\n=== Holdout ROI Correlation Between Runs ===')
roi_pivot = df_sniper.pivot(index='bet_type', columns='run', values='holdout_roi').dropna()
if len(roi_pivot) >= 3 and roi_pivot.shape[1] == 2:
    corr, pval = stats.pearsonr(roi_pivot[32], roi_pivot[33])
    print(f'Pearson correlation of holdout ROI (R32 vs R33): {corr:.3f} (p={pval:.3f})')
    print(f'Based on {len(roi_pivot)} bet types with holdout data in both runs')
else:
    print('Insufficient overlapping data for correlation')

In [ ]:
# Enable/disable recommendation
print('=== Market Enable/Disable Recommendations (Based on Run 33 Holdout) ===')
rec_rows = []
for bt in BET_TYPES:
    s33 = sniper[33].get(bt)
    s32 = sniper[32].get(bt)
    if s33 is None:
        continue
    h33 = s33.get('holdout_metrics', {})
    h32 = s32.get('holdout_metrics', {}) if s32 else {}
    roi33 = h33.get('roi', 0)
    prec33 = h33.get('precision', 0)
    roi32 = h32.get('roi', 0)
    
    if roi33 < 0 and roi32 < 0:
        status = 'DISABLE'
        reason = f'Negative ROI in both runs ({roi32:.1f}%, {roi33:.1f}%)'
    elif roi33 < 0:
        status = 'REVIEW'
        reason = f'Negative holdout ROI in R33 ({roi33:.1f}%), positive in R32 ({roi32:.1f}%)'
    elif prec33 < 0.55:
        status = 'CAUTION'
        reason = f'Precision below 55% ({prec33:.1%})'
    else:
        status = 'ENABLE'
        reason = f'ROI={roi33:.1f}%, Precision={prec33:.1%}'
    
    rec_rows.append({
        'bet_type': bt, 'status': status, 'reason': reason,
        'holdout_roi_r32': roi32, 'holdout_roi_r33': roi33,
        'holdout_precision_r33': prec33,
    })

df_market_rec = pd.DataFrame(rec_rows)
display(df_market_rec.sort_values('holdout_roi_r33', ascending=False))

## 6. Actionable Recommendations Summary

In [ ]:
print('=' * 80)
print('ACTIONABLE RECOMMENDATIONS FOR RUN 34')
print('=' * 80)

# 1. Feature param recommendations
print('\n--- FEATURE PARAMETER RANGE CHANGES ---')
flagged_33 = df_boundary[(df_boundary.run == 33) & (df_boundary.at_boundary != 'OK')]
if len(flagged_33) > 0:
    for _, row in flagged_33.iterrows():
        direction = 'lower' if row['at_boundary'] == 'LOW' else 'upper'
        current_bound = row['lo'] if direction == 'lower' else row['hi']
        suggested = int(current_bound * 0.7) if direction == 'lower' else int(current_bound * 1.4)
        print(f"  {row['bet_type']}/{row['param']}: Expand {direction} bound "
              f"from {current_bound} to ~{suggested} (value={row['value']})")
else:
    print('  No parameters at boundary in run 33 - search spaces are adequate.')

# 2. Model config
print('\n--- MODEL SELECTION PER BET TYPE ---')
for bt in BET_TYPES:
    s33 = sniper[33].get(bt)
    if s33 is None:
        continue
    model = s33.get('best_model', 'unknown')
    wf_best = s33.get('walkforward', {}).get('best_model_wf', model)
    match = 'CONSISTENT' if model == wf_best else f'MISMATCH (selected={model}, wf_best={wf_best})'
    print(f'  {bt}: {model} [{match}]')

# 3. Strategy decisions
print('\n--- MARKET STATUS RECOMMENDATIONS ---')
for _, row in df_market_rec.sort_values('holdout_roi_r33', ascending=False).iterrows():
    icon = {'ENABLE': '+', 'CAUTION': '~', 'REVIEW': '?', 'DISABLE': '-'}[row['status']]
    print(f"  [{icon}] {row['bet_type']}: {row['status']} - {row['reason']}")

# 4. Run 34 config suggestions
print('\n--- RUN 34 CONFIGURATION ---')
print('  Model trials: Keep at 75 (diminishing returns above this based on R32 vs R33 comparison)')
print('  Feature trials: Keep at 18 (convergence observed in most bet types)')

# Check if more trials helped
r32_avg_holdout = df_sniper[df_sniper.run == 32].holdout_roi.mean()
r33_avg_holdout = df_sniper[df_sniper.run == 33].holdout_roi.mean()
print(f'  R32 avg holdout ROI (115 trials): {r32_avg_holdout:.1f}%')
print(f'  R33 avg holdout ROI (75 trials): {r33_avg_holdout:.1f}%')
if r33_avg_holdout >= r32_avg_holdout:
    print('  -> 75 trials performed equal or better. No need to increase.')
else:
    diff = r32_avg_holdout - r33_avg_holdout
    print(f'  -> R32 outperformed by {diff:.1f}pp. Consider increasing trials back to 100+.')

# Feature removal suggestions
n_safe_removal = sum(1 for c in low_imp_counts.values() if c >= 7)
print(f'\n  Feature removal: {n_safe_removal} features are low-importance in 7+ bet types.')
print('  Consider removing these to reduce dimensionality and speed up training.')